In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [3]:
train_dir = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\train'
test_dir = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\test'

In [5]:
input_shape = (128, 128, 3)
batch_size = 32

In [7]:
data_generator = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [11]:
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [13]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\IT SHOP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\IT SHOP\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step


In [17]:
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [19]:
dt_classifier = DecisionTreeClassifier(
    criterion='gini',         
    max_depth=5)
dt_classifier.fit(train_features, train_labels)

DecisionTreeClassifier(max_depth=5)

In [21]:
test_predictions = dt_classifier.predict(test_features)

In [23]:
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 46.91%


In [25]:
model.save('cnn_feature_extractor4.h5')

In [27]:
joblib.dump(dt_classifier, 'dt_classifier.pkl')

['dt_classifier.pkl']

In [29]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_feature_extractor4.h5')
dt_classifier = joblib.load('dt_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for dt

    # Classify the features with the dt model
    prediction = dt_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\test\NonDemented\26 (86).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
The image is classified as: NonDemented
